In [1]:
import csv
import random
import importlib
from openai import OpenAI

import constants # Required for the following line if kernel is restarted
importlib.reload(constants) # Else the old key value is retained
from constants import openai_api

client = OpenAI(api_key=openai_api)

In [32]:
def attack_with_gpt(csv_filename):
    with open(csv_filename, mode='r') as file:
        reader = csv.reader(file)
        rows = list(reader)

    headers = rows[0]
    data = rows[1:]
    
    random.shuffle(data)
    shuffled_rows = [headers] + data
    
    formatted_data = "\n".join([", ".join(row) for row in shuffled_rows])

    prompt = f"""
    You are a query analysis assistant. Below is a CSV file containing rows of sequential queries.
    Each row corresponds to a different topic, and each column represents a sequential query flow.
    Only one of the rows is a geniune query flow made by a human user, the rest are AI generated
    to twart any adversary from figuring out what the actual search intention is. Your task is to 
    analyze the queries in each row and identify which one is the real, human-generated query. 
    Rank your predictions and return the top 10. Return them in a JSON format where the key is 
    the prediction rank and the value is the row index of the original query.

    CSV Data:
    {formatted_data}
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {"role": "system", "content": "You are a query analysis assistant."},
            {"role": "user", "content": prompt}
        ],
        response_format={ "type": "json_object" },
        temperature=0.7,
        seed=random.randint(0, 1000000)
    )

    result = response.choices[0].message.content
    return result

def evaluate_results(gpt_results):
    import json
    results = json.loads(gpt_results)

    for rank, row_index in results.items():
        if row_index == 0:
            print(f"Original query was detected as rank {rank}")
            return int(rank)
    return -1

def attack_and_eval(n_attacks, csv_filename):
    success_rank_sum = 0
    success_count_sum = 0
    for i in range(n_attacks):
        gpt_result = attack_with_gpt(csv_filename)
        rank = evaluate_results(gpt_result)
        if rank > 0:
            success_rank_sum += rank
            success_count_sum += 1

    success_rank_sum = success_rank_sum / success_count_sum
    success_count_sum = success_count_sum / n_attacks
    print(f"Average success rank: {success_rank_sum}")
    print(f"Average success count: {success_count_sum}")

    return success_rank_sum, success_count_sum

In [33]:
attack_and_eval(n_attacks=10, csv_filename="outputs/google_trends_r10_c3_p2.csv")

Original query was detected as rank 1
Original query was detected as rank 10
Original query was detected as rank 4
Original query was detected as rank 2
Original query was detected as rank 6
Original query was detected as rank 1
Original query was detected as rank 8
Original query was detected as rank 5
Original query was detected as rank 10
Original query was detected as rank 8
Average success rank: 5.5
Average success count: 1.0


(5.5, 1.0)

In [34]:
import glob

results = {}
all_csv_files = glob.glob("outputs/*_p2.csv")
for fn in all_csv_files:
    success_rank_sum, success_count_sum = attack_and_eval(n_attacks=30, csv_filename=fn)
    results[fn] = (success_rank_sum, success_count_sum)
print(results)

Original query was detected as rank 1
Original query was detected as rank 5
Original query was detected as rank 3
Original query was detected as rank 10
Original query was detected as rank 4
Original query was detected as rank 7
Original query was detected as rank 4
Original query was detected as rank 8
Original query was detected as rank 1
Original query was detected as rank 4
Original query was detected as rank 1
Original query was detected as rank 7
Original query was detected as rank 1
Original query was detected as rank 1
Original query was detected as rank 1
Original query was detected as rank 1
Original query was detected as rank 2
Original query was detected as rank 1
Original query was detected as rank 8
Original query was detected as rank 3
Original query was detected as rank 1
Original query was detected as rank 4
Original query was detected as rank 6
Original query was detected as rank 2
Original query was detected as rank 1
Original query was detected as rank 4
Original qu